In [1]:
# !python3 segment/predict.py \
#         --weights runs/train-seg/exp10/weights/best.pt \
#         --source "data/test/images"   # your own video/image

In [2]:
from segment.predict2 import Infer_seg
import numpy as np
import cv2
source = "data/test/images/203_trunk_png.rf.f78685c5f5f80468d9e52929278da43a.jpg"
weight_src = "runs/train-seg/exp10/weights/best.pt"

model = Infer_seg(weights=weight_src)
det_bbox, proto = model.forward(cv2.imread(source))

print(proto.shape)

YOLOv5 🚀 yolov7-segmentation-45-gb59e484 Python-3.10.16 torch-2.6.0+cu124 CUDA:0 (NVIDIA GeForce RTX 3090, 24250MiB)

Fusing layers... 
Model summary: 325 layers, 37847870 parameters, 0 gradients, 141.9 GFLOPs


torch.Size([4, 640, 640])


In [3]:
import matplotlib.pyplot as plt
import matplotlib
matplotlib.use('TkAgg')
import cv2
print(proto.shape)
cv2.imshow('frame',proto.detach().cpu().numpy()[2])
cv2.waitKey(0)
cv2.destroyAllWindows()

torch.Size([4, 640, 640])


In [4]:
# the detections are splits into proto aka the masks of each detection
# 
print(proto.detach().cpu().numpy()[1].shape)

(640, 640)


In [7]:
import torch
def im_mask_from_cls(detection_bbox,im_mask, cls=1):
    x1,y1,x2,y2,conf,detclass = detection_bbox[:,:6].T
    class_indices = torch.where(detclass == cls)[0]
    cls_mask = torch.clamp(im_mask[class_indices].sum(dim=0), 0,1)
    return cls_mask
    
im_mask = im_mask_from_cls(det_bbox, proto)
cv2.imshow('frame',im_mask.detach().cpu().numpy())
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
def im_mask_from_center_region(detection_bbox, im_mask, cls=1, center_tol=200):
    """
    Computes a mask for detections of class `cls` within `center_tol` pixels of the image center.
    
    Args:
        detection_bbox (Tensor): [N, 6] (x1, y1, x2, y2, conf, class)
        im_mask (Tensor): [N, H, W] (binary masks per detection)
        cls (int): Class ID to filter
        center_tol (int): Pixel tolerance around center
    
    Returns:
        Tensor: [H, W] binary mask (summed & clamped to [0, 1])
    """
    # Extract bbox data [N, 6] -> (x1, y1, x2, y2, conf, class)
    x1, y1, x2, y2, conf, detclass = detection_bbox[:, :6].T  # shapes: [N]
    H, W = im_mask.shape[1], im_mask.shape[2]
    
    # Compute center bounds (fixed region)
    center_x, center_y = W // 2, H // 2
    x_min, x_max = center_x - center_tol, center_x + center_tol
    y_min, y_max = center_y - center_tol, center_y + center_tol
    
    # Find detections of class `cls` whose centers are inside the central region
    box_centers_x = (x1 + x2) / 2  # [N]
    box_centers_y = (y1 + y2) / 2  # [N]
    
    is_cls = (detclass == cls)  # [N]
    is_in_center = (
        (box_centers_x >= x_min) & 
        (box_centers_x <= x_max) & 
        (box_centers_y >= y_min) & 
        (box_centers_y <= y_max))
    
    valid_indices = torch.where(is_cls & is_in_center)[0]
    
    if len(valid_indices) == 0:
        return torch.zeros_like(im_mask[0]), 0 , (0,0) # No matches
    elif len(valid_indices) > 1:
        valid_indices = valid_indices[torch.argmax(conf[valid_indices])].unsqueeze(0)
    else: # Valid indices is only 1
        pass
    # Sum all valid masks and clamp
    cls_mask = torch.clamp(im_mask[valid_indices].sum(dim=0), 0, 1)
    return cls_mask.detach().cpu().numpy(), len(valid_indices), (box_centers_x[valid_indices], box_centers_y[valid_indices])

im_mask, det = im_mask_from_center_region(det_bbox, proto, center_tol=200)
print(im_mask.shape)
cv2.imshow('frame',im_mask)
cv2.waitKey(0)
cv2.destroyAllWindows()

(640, 640)


In [15]:
print(det)

1
